In [ ]:
%load_ext autoreload
%autoreload 2

# Imports and definitions

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from datetime import datetime
import xarray as xr

In [ ]:
from emg_from_lfp import compute_emg
from ecephys_analyses.data import channel_groups, paths
from ecephys.sglx_utils import load_timeseries

In [ ]:
EMG_OPTS = dict(target_sf=20, window_size=25.0, wp=[300, 600], ws=[275, 625], gpass=1, gstop=60, ftype='butter')

In [ ]:
def run_emg_pipeline_on_file(bin_path, emg_path, emg_chans):
    emg_sig = load_timeseries(bin_path, emg_chans)
    emg_values = compute_emg(emg_sig.values.T, emg_sig.fs, **EMG_OPTS).T
    emg_times = np.linspace(np.min(emg_sig.time), np.max(emg_sig.time), len(emg_values))
    emg = xr.DataArray(emg_values.flatten(), dims="time", coords={"time": emg_times})
    emg.attrs['long_name'] = 'EMG'
    emg.attrs['units'] = 'zero-lag correlation'
    emg.attrs['file_start'] = emg_sig.file_create_time 
    for key in EMG_OPTS:
        emg.attrs[key] = EMG_OPTS[key]
    emg.to_netcdf(emg_path)
    emg.close()

In [ ]:
def run_emg_pipeline_on_condition(subject, condition):
    emg_chans = channel_groups.emg[subject]
    
    bin_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="lf.bin")
    emg_paths = paths.get_sglx_style_datapaths(subject=subject, condition=condition, ext="emg.nc")

    for bin_path, emg_path in zip(bin_paths, emg_paths):
        run_emg_pipeline_on_file(bin_path, emg_path, emg_chans)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"{current_time}: Finished {str(bin_path)}")

# Run automated pipeline

### Valentino

In [ ]:
run_emg_pipeline_on_condition(subject="Segundo", condition="recovery-sleep-6h")

### Valentino

In [ ]:
run_emg_pipeline_on_condition(subject="Valentino", condition="recovery-sleep-6h")

### Doppio

In [ ]:
run_emg_pipeline_on_condition(subject="Doppio", condition="sleep-deprivation-12h-circadian-match")

nChan: 385, nFileSamp: 18000019


In [ ]:
run_emg_pipeline_on_condition(subject="Doppio", condition="recovery-sleep-6h")

### Alessandro

In [ ]:
run_emg_pipeline_on_condition(subject="Alessandro", condition="extended-wake")

In [ ]:
run_emg_pipeline_on_condition(subject="Alessandro", condition="recovery-sleep-6h")

### Eugene

In [ ]:
run_emg_pipeline_on_condition(subject="Eugene", condition="extended-wake")

In [ ]:
run_emg_pipeline_on_condition(subject="Eugene", condition="recovery-sleep-6h")

# Run pipeline piecemeal

In [ ]:
import matplotlib.pyplot as plt
from neurodsp.plts.time_series import plot_time_series
from ecephys.scoring import load_visbrain_hypnogram
from ecephys.plot import plot_hypnogram_overlay

In [ ]:
SUBJECT = "Segundo"
CONDITION = "REC-2-0"
START_TIME = 0
END_TIME = None
bin_path = paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="lf.bin")

In [ ]:
emg_chans = channel_groups.emg[SUBJECT]
emg_sig = load_timeseries(bin_path, emg_chans, start_time=START_TIME, end_time=END_TIME, xarray=True)

In [ ]:
emg_opts = dict(target_sf=20, window_size=25.0, wp=[300, 600], ws=[275, 625], gpass=1, gstop=60, ftype='butter')
emg_values = compute_EMG(emg_sig.values.T, emg_sig.fs, **emg_opts).T

In [ ]:
emg_times = np.linspace(np.min(emg_sig.time), np.max(emg_sig.time), len(emg_values)) + START_TIME

In [ ]:
emg = xr.DataArray(emg_values.flatten(), dims="time", coords={"time": emg_times})
emg.attrs['long_name'] = 'EMG'
emg.attrs['units'] = 'zero-lag correlation'
for key in emg_opts:
    emg.attrs[key] = emg_opts[key]

In [ ]:
_, ax = plt.subplots(figsize=(24, 3))
plot_time_series(emg.time.values, emg.values, ax=ax)
#plot_hypnogram_overlay(hypno, ax=ax)

### Save

In [ ]:
emg_ds = xr.Dataset({"emg": emg})

In [ ]:
emg_ds.to_netcdf(paths.get_datapath(subject=SUBJECT, condition=CONDITION, data="emg.nc"))

In [ ]:
emg_ds.close()